In [1]:
!git clone https://github.com/ZhengPeng7/BiRefNet.git
!pip uninstall -q torchaudio torchdata torchtext -y
!pip install -q -r BiRefNet/requirements.txt
!pip install -q -U gdown
!wget https://github.com/ZhengPeng7/BiRefNet/releases/download/v1/BiRefNet-COD-epoch_125.pth
!gdown 1vRYAie0JcNStcSwagmCq55eirGyMYGm5 && unzip -q COD10K-v3.zip && rm COD10K-v3.zip
%cd BiRefNet

Cloning into 'BiRefNet'...
remote: Enumerating objects: 1376, done.
remote: Counting objects: 100% (742/742), done.
remote: Compressing objects: 100% (236/236), done.
remote: Total 1376 (delta 596), reused 622 (delta 504), pack-reused 634 (from 1)
Receiving objects: 100% (1376/1376), 5.44 MiB | 39.79 MiB/s, done.
Resolving deltas: 100% (941/941), done.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 2.4.0 requires torch>=2.1.0, but you have torch 2.0.1+cu118 which is incompatible.
--2024-11-08 20:53:06--  https://github.com/ZhengPeng7/BiRefNet/releases/download/v1/BiRefNet-COD-epoch_125.pth
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/

In [2]:
with open('models/birefnet.py', 'r') as f:
    content = f.read()
    
with open('models/birefnet.py', 'w') as f:
    content_new = content.replace(', pretrained=True)', ', pretrained=False)')
    f.write(content_new)

with open('config.py', 'r') as f:
    content = f.read()
    
with open('config.py', 'w') as f:
    content_new = content.replace("self.data_root_dir = os.path.join(self.sys_home_dir, 'datasets/dis')", "self.data_root_dir = '..'")
    content_new = content_new.replace('self.load_all = True', 'self.load_all = False')
    content_new = content_new.replace('self.num_workers = 5', 'self.num_workers = 1')
    f.write(content_new)

In [3]:
import os
from glob import glob

class ParserTest():
    def __init__(self) -> None:
        self.ckpt = '../BiRefNet-COD-epoch_125.pth'
        self.ckpt_folder = ''
        self.pred_root = '../e_preds'
        self.testsets = 'COD10k-v3'

args_test = ParserTest()

In [4]:
%cd /kaggle/working

/kaggle/working


In [5]:
# !rm -rf DIS5K

# !rm -rf COD10K-v3

In [6]:
!mkdir -p "DIS5K/COD10K-v3"
!cp -r COD10K-v3/* DIS5K/COD10K-v3
!rm -rf COD10K-v3
%cd DIS5K/COD10K-v3
!mkdir im gt
!cp -r Test/Image/* im
!cp -r Test/GT_Object/* gt
!rm -rf Test
%cd /kaggle/working/BiRefNet

/kaggle/working/DIS5K/COD10K-v3
/kaggle/working/BiRefNet


In [7]:
from inference import main as inference_main
CUSTOM_IMAGE_FOLDER = ''

for testset in 'COD10K-v3+{}'.format(CUSTOM_IMAGE_FOLDER).strip('+').split('+'):
    args_test.testsets = testset
    inference_main(args_test)

Testing with model ../BiRefNet-COD-epoch_125.pth
>>>> Testset: COD10K-v3...
	Inferencing ../BiRefNet-COD-epoch_125.pth...


/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
100%|██████████| 4000/4000 [1:10:38<00:00,  1.06s/it]


In [8]:
from eval_existingOnes import do_eval
from config import Config

config = Config()
class ParserEval():
    def __init__(self) -> None:
        self.gt_root = os.path.join(config.data_root_dir, config.task)
        self.pred_root = '../e_preds'
        self.data_lst = {
            'DIS5K': '+'.join(['COD10K-v3'][:1]),
            # 'COD': '+'.join(['COD10K-v3', 'NC4K', 'CAMO', 'CHAMELEON'][:1]),
            # 'SOD': '+'.join(['DAVIS-S', 'HRSOD-TE', 'UHRSD-TE', 'DUTS-TE', 'DUT-OMRON'][:1])
        }[config.task]
        self.model_lst = [args_test.ckpt] if args_test.ckpt else sorted(glob(os.path.join(args_test.ckpt_folder, '*')))[-1]
        self.save_dir = '../e_results'
        self.metrics = '+'.join(['S', 'BIoU', 'MAE', 'E', 'F', 'WF', 'HCE'][:7])
args_eval = ParserEval()
os.makedirs(args_eval.save_dir, exist_ok=True)
args_eval.model_lst = [m for m in sorted(os.listdir(args_eval.pred_root), key=lambda x: int(x.split('epoch_')[-1]))]
do_eval(args_eval)

#################### COD10K-v3 ####################
	 Evaluating model: --BiRefNet-COD-epoch_125...


100%|██████████| 4000/4000 [16:31<00:00,  4.04it/s]

+-----------+--------------------------+-------+-----------+------+----------+--------+------+-------+--------+-------+-------+------+---------+----------+
&  Dataset  &          Method          & maxFm & wFmeasure & MAE  & Smeasure & meanEm & HCE  & maxEm & meanFm & adpEm & adpFm & mBA  & maxBIoU & meanBIoU &
+-----------+--------------------------+-------+-----------+------+----------+--------+------+-------+--------+-------+-------+------+---------+----------+
& COD10K-v3 & --BiRefNet-COD-epoch_125 &  .454 &    .443   & .078 &   .885   &  .908  & 67   &  .912 &  .450  &  .795 &  .445 & .000 &   .376  &   .369   &
+-----------+--------------------------+-------+-----------+------+----------+--------+------+-------+--------+-------+-------+------+---------+----------+
